In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [ ]:
!wget https://github.com/katearb/files/raw/main/titanic_train.csv

--2022-06-01 10:13:51--  https://github.com/katearb/files/raw/main/titanic_train.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/katearb/files/main/titanic_train.csv [following]
--2022-06-01 10:13:51--  https://raw.githubusercontent.com/katearb/files/main/titanic_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic_train.csv’

titanic_train.csv   100%[===================>]  58.89K  --.-KB/s    in 0.001s  

2022-06-01 10:13:51 (55.8 MB/s) - ‘titanic_train.csv’ saved [60302/60302]



In [ ]:
df = pd.read_csv('/content/titanic_train.csv')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [ ]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [ ]:
x = df.drop('Survived', axis=1)
y = df['Survived']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
si_age=SimpleImputer()
si_emb=SimpleImputer(strategy='most_frequent')

x_train_age=si_age.fit_transform(x_train[['Age']])
x_train_embarked=si_emb.fit_transform(x_train[['Embarked']])

x_test_age=si_age.transform(x_test[['Age']])
x_test_embarked=si_emb.transform(x_test[['Embarked']])

In [ ]:
ohe_sex=OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked=OneHotEncoder(sparse=False,handle_unknown='ignore')

x_train_sex=ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked=ohe_embarked.fit_transform(x_train_embarked)

x_test_sex=ohe_sex.transform(x_test[['Sex']])
x_test_embarked=ohe_embarked.transform(x_test_embarked)

In [ ]:
x_train_rem=x_train.drop(columns=['Sex','Age','Embarked'])
x_test_rem=x_test.drop(columns=['Sex','Age','Embarked'])

In [ ]:
x_train_trans=np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis=1)
x_test_trans=np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis=1 )

In [ ]:
x_test_trans.shape


(179, 10)

In [ ]:
clf=DecisionTreeClassifier()
clf.fit(x_train_trans,y_train)

DecisionTreeClassifier()

In [ ]:
y_pred=clf.predict(x_test_trans)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7877094972067039

In [ ]:
import pickle

In [ ]:
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))

PRODUCTION CODE / CODE FOR OUTPUT:

In [ ]:
ohe_sex=pickle.load(open('ohe_sex.pkl','rb'))
ohe_embarked=pickle.load(open('ohe_embarked.pkl','rb'))
clf=pickle.load(open('clf.pkl','rb'))

In [ ]:
test_input=np.array([2,'male',31.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [ ]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [ ]:
test_input_age=test_input[:,2].reshape(1,1)

In [ ]:
test_input_sex=ohe_sex.transform(test_input[:,1].reshape(1,1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
test_input_embarked=ohe_embarked.transform(test_input[:,-1].reshape(1,1))

In [ ]:
test_input_trans=np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [ ]:
test_input_trans.shape

(1, 10)

In [ ]:
clf.predict(test_input_trans)

array([1])